In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### ResNet -50 implementaion with transfer learning

In [ ]:
#importing Libraries
from tensorflow import keras
from tensorflow.keras import layers
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Model

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255, zoom_range=0.3, rotation_range=50,
 width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2, 
 horizontal_flip=True, fill_mode='nearest')

test_datagen = ImageDataGenerator(
    rescale = 1/255
)

In [ ]:
from keras.layers import Input
model = ResNet50(include_top= False,input_shape=  (224,224,3))

94781440/94765736 [==============================] - 1s 0us/step


In [ ]:
for layer in model.layers:
  layer.trainable =False

In [ ]:
flatten_layer = layers.Flatten()(model.output)
flattened_fc_layer = layers.Dense(512, activation='relu')(flatten_layer)
flattened_fc_layer_2 = layers.Dense(1024, activation='relu')(flattened_fc_layer)
flattened_fc_softmax_layer = layers.Dense(3, activation='softmax')(flattened_fc_layer_2)

In [ ]:
model = Model(inputs = model.inputs, outputs = flattened_fc_softmax_layer)

In [ ]:
training_iterator = train_datagen.flow_from_directory('/content/drive/MyDrive/DL_proj_dataset/train', batch_size=30, target_size=(224,224))
testing_iterator = test_datagen.flow_from_directory('/content/drive/MyDrive/DL_proj_dataset/test', batch_size=30, target_size=(224,224))

Found 1148 images belonging to 3 classes.
Found 286 images belonging to 3 classes.


In [ ]:
training_iterator.class_indices

{'medicinal': 0, 'parasitic': 1, 'poisonous': 2}

In [ ]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
history = model.fit(training_iterator, validation_data = testing_iterator, epochs=10)

Epoch 1/10
39/39 [==============================] - 205s 5s/step - loss: 3.3407 - accuracy: 0.6768 - val_loss: 0.6339 - val_accuracy: 0.8881
Epoch 2/10
39/39 [==============================] - 48s 1s/step - loss: 0.3928 - accuracy: 0.8720 - val_loss: 0.2395 - val_accuracy: 0.9056
Epoch 3/10
39/39 [==============================] - 48s 1s/step - loss: 0.2782 - accuracy: 0.8902 - val_loss: 0.2562 - val_accuracy: 0.9056
Epoch 4/10
39/39 [==============================] - 47s 1s/step - loss: 0.2747 - accuracy: 0.8841 - val_loss: 0.1950 - val_accuracy: 0.9266
Epoch 5/10
39/39 [==============================] - 49s 1s/step - loss: 0.2245 - accuracy: 0.9016 - val_loss: 0.2421 - val_accuracy: 0.9091
Epoch 6/10
39/39 [==============================] - 50s 1s/step - loss: 0.2577 - accuracy: 0.8929 - val_loss: 0.2225 - val_accuracy: 0.8986
Epoch 7/10
39/39 [==============================] - 48s 1s/step - loss: 0.2706 - accuracy: 0.8929 - val_loss: 0.2799 - val_accuracy: 0.8846
Epoch 8/10
39/39 [=

In [ ]:
from tensorflow.keras.preprocessing import image 
import numpy as np
img= image.load_img("test-1.jpg",target_size=(224,224))
#Converting the test image from "PIL.Image.Image" format to "numpy.ndarray" format
x=image.img_to_array(img)
type(x)
#Increasing the no.of dimensions of the image array, so as to fit in the model.
x=np.expand_dims(x,axis=0)
x.shape
#Predicting the class of x
y_pred= np.argmax(model.predict(x))
y_pred

In [ ]:
type(img)

PIL.Image.Image

In [ ]:
model.save("model1.h5")

/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


### DenseNet implementaion with transfer learning

In [ ]:
from tensorflow.keras.applications.densenet import DenseNet121

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255, zoom_range=0.3, rotation_range=50,
 width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2, 
 horizontal_flip=True, fill_mode='nearest')

test_datagen = ImageDataGenerator(
    rescale = 1/255
)

In [ ]:
training_iterator = train_datagen.flow_from_directory('/content/drive/MyDrive/DL_proj_dataset/train', batch_size=30, target_size=(64,64))
testing_iterator = test_datagen.flow_from_directory('/content/drive/MyDrive/DL_proj_dataset/test', batch_size=30, target_size=(64,64))

In [ ]:
def build_densenet():
    densenet = DenseNet121(weights='imagenet', include_top=False)

    input = Input(shape=(64, 64, 3))
    x = Conv2D(3, (3, 3), padding='same')(input)
    
    x = densenet(x)
    
    x = AveragePooling2D()(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    x = Dense(256, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)

    # multi output
    output = Dense(15,activation = 'softmax', name='root')(x)
 

    # model
    model = Model(input,output)
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()
    
    return model


In [ ]:
from keras import layers
from keras.layers import AveragePooling2D
from tensorflow.keras.layers import BatchNormalization
from keras.layers import Dropout
from keras.layers import Dense

model = build_densenet()
datagen = ImageDataGenerator(rotation_range=360, # Degree range for random rotations
                        width_shift_range=0.2, # Range for random horizontal shifts
                        height_shift_range=0.2, # Range for random vertical shifts
                        zoom_range=0.2, # Range for random zoom
                        horizontal_flip=True, # Randomly flip inputs horizontally
                        vertical_flip=True) # Randomly flip inputs vertically

datagen.fit(training_iterator)
# Fits the model on batches with real-time data augmentation
hist = model.fit_generator(datagen.flow(training_iterator, Y_train, batch_size=BATCH_SIZE),
               steps_per_epoch=training_iterator.shape[0] // BATCH_SIZE,
               epochs=10,
               verbose=2,
               validation_data=testing_iterator)